# Adversarial Learning on Malware Data
Data derived from: https://figshare.com/articles/Android_malware_dataset_for_machine_learning_1/5854590/1

The first few cells are dedicated to processing data

In [1]:
using DataFrames
using CSV
using Statistics
using Knet: Knet, KnetArray, gpu, minibatch
using StatsBase
import Missings
using Optim
using Knet: SGD, train!, nll, zeroone, relu, dropout
using AutoGrad: Param
import ProgressMeter

function remove_missing!(df)
    for col in names(df)
        df[col] = Missings.coalesce.(df[col], 0)
    end
end

function process_binary_data(data, train_rat)
    """
    Takes in data in the form of a DataFrame with a :class
    column containing either 1 or 2 (binary classification)
    Returns a training set, a testing set, and all entries
    in class 2 (for use in permuting them adversarially)
    """
    suspicious = data[data[:class] .== 2, :]
    benign = data[data[:class] .== 1, :]

    train_benign_count = Int(round(size(benign)[1]*train_rat))
    train_susp_count = Int(round(size(suspicious)[1]*train_rat))

    s_train_inds = sample(1:size(suspicious)[1], train_susp_count, replace=false)
    b_train_inds = sample(1:size(benign)[1], train_benign_count, replace=false)
    s_test_inds = setdiff(1:size(suspicious)[1], s_train_inds)
    b_test_inds = setdiff(1:size(benign)[1], b_train_inds)

    s_train = suspicious[s_train_inds, :]
    s_test = suspicious[s_test_inds, :]
    b_train = benign[b_train_inds, :]
    b_test = benign[b_test_inds, :]

    trn = [s_train; b_train]
    remove_missing!(trn)
    tst = [s_test; b_test]
    remove_missing!(tst)

    shuffle = sample(1:size(trn)[1], size(trn)[1], replace=false)
    dtrn = Matrix(trn[shuffle, :])'
    trnx, trny = dtrn[1:end - 1, :], dtrn[end, :]
    # trny = hcat(trny, -trny .+ 1)'
    dtrn = minibatch(trnx, trny, 100)
    shuffle = sample(1:size(tst)[1], size(tst)[1], replace=false)
    dtst = Matrix(tst[shuffle, :])'
    tstx, tsty = dtst[1:end - 1, :], dtst[end, :]
    # tsty = hcat(tsty, -tsty .+ 1)'
    dtst = minibatch(tstx, tsty, 100)

    remove_missing!(suspicious)
    susp = Matrix(suspicious)'[1:end-1, :]
    return dtrn, dtst, susp
end

process_binary_data (generic function with 1 method)

In [2]:
data = CSV.read("malgenome-215-dataset-1260malware-2539-benign.csv")
bin = Dict("B" => 1, "S" => 2)
data[:class] = map(elt->bin[elt], data[:class])

dtrn, dtst, susp = process_binary_data(data, 0.7)

(Knet.Data([1 0 … 1 1; 1 0 … 1 1; … ; 0 0 … 0 0; 0 0 … 0 0], [1 1 … 1 1], 100, 2659, false, 1:2659, false, (215, 2659), (2659,), Array{Int64,2}, Array{Int64,1}), Knet.Data([1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 0; 0 0 … 0 0], [1 2 … 1 2], 100, 1140, false, 1:1140, false, (215, 1140), (1140,), Array{Int64,2}, Array{Int64,1}), [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

# Defining an MLP to attack

Code primarily derived from the Knet tutorial

In [3]:
function softmax(x::Array{Float32, 1})
    ret = exp.(x)
    ret / sum(ret)
end

function softmax(x::Array{Float64, 1})
    ret = exp.(x)
    ret / sum(ret)
end

function softmax(x::Array{Float32, 2}; dims=1)
    mapslices(softmax, x; dims=dims)
end

function trainresults(file,model; o...)
    if (print("Train from scratch? ");readline()[1]=='y')
        results = Float64[]; updates = 0; prog = ProgressMeter.Progress(60000)
        function callback(J)
            if updates % 600 == 0
                push!(results, nll(model,dtrn), nll(model,dtst), zeroone(model,dtrn), zeroone(model,dtst))
                ProgressMeter.update!(prog, updates)
            end
            return (updates += 1) <= 60000
        end
        train!(model, dtrn; callback=callback, optimizer=SGD(lr=0.1), o...)
        Knet.save(file,"results",reshape(results, (4,:)))
    end
    results = Knet.load(file,"results")
    println(minimum(results,dims=2))
    return results
end

struct Linear; w; b; end
(f::Linear)(x) = (f.w * x .+ f.b)

Linear(inputsize::Int,outputsize::Int) = Linear(param(outputsize,inputsize),param0(outputsize))
param(d...; init=xavier, atype=atype())=Param(atype(init(d...)))
param0(d...; atype=atype())=param(d...; init=zeros, atype=atype)
xavier(o,i) = (s = sqrt(2/(i+o)); 2s .* rand(o,i) .- s)
atype()=(gpu() >= 0 ? KnetArray{Float32} : Array{Float32})

struct MLP; layers; end
MLP(h::Int...)=MLP(Linear.(h[1:end-1], h[2:end]))

function (m::MLP)(x; pdrop=0, smax=false)
    for (i,layer) in enumerate(m.layers)
        p = (i <= length(pdrop) ? pdrop[i] : pdrop[end])
        x = dropout(x, p)     # <-- This one line helps generalization
        x = layer(x)
        x = (layer == m.layers[end] ? x : relu.(x))
    end
    if smax
        return softmax(x)
    else
        return x
    end
end

function accuracy(m)
    correct = 0.0
    wrong = 0.0
    for (x,y) in dtst
        pred = model(x; smax=true)
        inds = argmax(pred, dims=1)
        for (i, ind) in enumerate(inds)
            if ind[1] == y[i]
                correct += 1
            else
                wrong += 1
            end
        end
    end
    correct / (correct + wrong)
end

accuracy (generic function with 1 method)

# Training the model on malware data
Though the model is relatively simple, and the data set small, it acheives 98.8% accuracy on the testing set

In [ ]:
model = MLP(215, 64, 2)
println(accuracy(model))
mlp = trainresults("mlp_trial.jld2", model)
println(accuracy(model))

0.37363636363636366
Train from scratch? stdin> yes


Progress:  21%|█████████                                |  ETA: 0:04:04

# First Adversarial Attempt

Since the data for each field is a binary 0 or 1, I began by searching the space of all adjacent (differing only in 1 feature) possible feature vectors to find an adversarial example. This could be put into practice by adding or removing a single API call from the malware to cause this neural net to misclassify it

In [ ]:
function near_search(x0; model=model)
    best_x = nothing
    best_score = 0
    for (i, x) in enumerate(x0)
        if x == 0
            new_x = x0
            new_x[i] = 1 - x
            pred = model(new_x, smax=true)
            if pred[1] > pred[2] && pred[1] > best_score
                best_x = new_x
                best_score = pred[1]
            end
        end
    end
    return best_x
end

In [ ]:
total = size(susp,2)
successes = 0
for i in 1:size(susp,2)
    x0 = susp[:,i]
    pred0 = model(x0, smax=true)
    x1 = near_search(x0)  
    pred = model(x1, smax=true)
    if pred[1] > pred[2]
        successes += 1
    end
end
successes / total

As demonstrated above, for every single suspicious application, I found an adjacent application that the model will classify as benign. For some next steps, consider adding dropout to the trained model and see if that affects its susceptibility to this simple attack, and perhaps try defensive distillation, as documented in "Distillation as a defense to adversarial perturbations against deep neural networks" by Papernot et al (2016). To strengthen the attack, consider allowing larger perturbations or following gradients to find perturbation. Once the attack is sufficiently strong, I can try adversarial learning, where I train the model on the data, generate adversarial examples, then train it on the adversarial examples.

For a parallel problem, consider adversarial examples for mnist data (code derived from the tutorial)

In [13]:
include(Knet.dir("data","mnist.jl"))
xsize=784
ysize=10
dtrn,dtst = mnistdata(xsize=(xsize,:))

model = MLP(784,64,10)
accuracy(model)
mlp2 = trainresults("mlp2.jld2", model);
accuracy(model)

Train from scratch? stdin> y


Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:01

[0.00609881; 0.0857389; 0.0005; 0.0248]


Progress: 100%|█████████████████████████████████████████| Time: 0:01:36


0.9749

In [43]:
examples = []
for (x,y) in dtst
    if y != 1
        push!(examples, (x,y))
    end
end
examples[1][2][1]

0x07

In [51]:
using Distances
using BlackBoxOptim

# This x0 is supposed to be a 7
x0 = examples[1][1][:, 1]

function error(xp; model=model, c=1, x=x0, target=1)
    """
    Computes a value to be minimized when generating adversarial examples
    model is the model to fool
    c is a parameter to tun how much to weigh distance from the original
    x0 is the original
    target is the target class
    
    See "Intriguing properries of neural networks" Szegedy et al (2013)
    """
    c * euclidean(x, xp)^2 - log(model(xp, smax=true)[target])
end

println(model(x0, smax=true))
x1 = bboptimize(error; SearchRange = (0.0, 1.0), NumDimensions = 784).archive_output.best_candidate
println(model(x1, smax=true))
println(euclidean(x1, x0))

Float32[1.09696e-11, 1.49714e-6, 1.5021e-7, 6.9651e-12, 8.79741e-10, 1.82453e-18, 0.999997, 2.12942e-8, 1.47317e-6, 2.45452e-10]
Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps

Optimization stopped after 10001 steps and 0.29200005531311035 seconds
Termination reason: Max number of steps (10000) reached
Steps per second = 34249.99351207647
Function evals per second = 34571.91125931527
Improvements/step = 0.2182
Total function evaluations = 10095


Best candidate found: [0.821732, 0.269588, 0.557876, 0.466721, 0.725955, 0.303658, 0.341389, 0.375764, 0.338666, 0.244843, 0.512377, 0.363907, 0.468888, 0.270758, 0.596601, 0.322219, 0.366059, 0.377868, 0.639013, 0.385687, 0.674143, 0.520954, 0.489032, 0.510428, 0.401437, 0.381548, 0.354966, 0.410504, 0.496946, 0.333049, 0.417577, 0.132851, 0.374933, 0.361233, 0.263818, 0.124261, 0.147817, 0.588

[0.49008, 0.115383, 0.0920092, 3.29477e-5, 0.0628615, 0.000633907, 1.35619e-5, 0.120905, 0.00306327, 0.115017]
11.544430411038785


Before adversarial noise was added, the model (correctly) predicted that the input was a 7 with 99.9% confidence. After perturbation, it is most confident (49%) that the input is a 1, despite the two being only 11.5 apart in terms of Euclidean distance.  
This method can be refined through use of a line-search to find the value of c that minimizes the distance of the adversarial example from the original while still fooling the model.  
(I would also like to find a way to disable all that output from bboptimize)

In [ ]:
function binary_search_old(f; max_depth=5, max_x=100)
    """
    Uses a binary search to minimize f
    Needs some updating. If it hasn't hit the target, search down
    Otherwise search up
    """
    curr_x = max_x / 4
    curr_diff = max_x / 2
    cont = true
    for i in 1:max_depth
        val_l = f(curr_x)
        val_r = f(curr_x + curr_diff)
        curr_x = val_l < val_r ? curr_x - curr_diff / 4 : curr_x + 3 * curr_diff / 4
        curr_diff = curr_diff / 2
    end
    return curr_x
end

In [7]:
s = "龙abc"

'~': ASCII/Unicode U+007e (category Sm: Symbol, math)